# LLM tooling for Chemistry

<p style="font-style: italic; font-size: 18px; color: dimgray;">ChatGPT and Gemini</p>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2025

-----

## Introduction

This notebook shows how to do parallel [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini), [AAp2], is used.

-----

## Setup

Load packages:

In [1]:
use JSON::Fast;
use Data::Reshapers;
use Data::TypeSystem;
use LLM::Functions;
use LLM::Functions::TooledGemini;
use LLM::Graph;
use Chemistry::Stoichiometry;
use WWW::Gemini;

Choose a model:

In [2]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', :8192max-tokens, temperature => 0.4);
sink my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash', :8192max-tokens, temperature => 0.4);

----

## Diagram

Here is a [Mermaid-JS](https://mermaid.js.org) diagram that shows _single-pass_ LLM-and-tool interaction:


```mermaid
sequenceDiagram
    participant Developer
    participant Model as LLM Service (Gemini)
    participant Tool as External Function

    Note over Developer, Model: 1. Tool Definitions + Messages
    Developer->>Model: molecular-mass(molecule) <br> "What's the mass of SO2?"

    Note over Model: 2. Tool Calls
    Model->>Tool: molecular-mass("SO2")

    Note over Tool: 3. Execute Function Code
    Tool->>Model: {"mass": 64.058}

    Note over Model: 4. Results
    Model->>Developer: {"mass": 64.058}

    Note over Developer: 5. Final Response
    Developer->>Model: The molecular mass of SO2 is 64.058.
```


**Remark:** Instead of one invocation of a local tool -- as shown in the diagram -- often enough multiple LLM-&-local-computer interactions are used for LLM-tool computation.


---

## Chemistry computations examples

Here is an example of calling `molecular-mass`:

In [3]:
molecular-mass("SO2")

64.058

----

## Define package functions as tools

Define a few subs -- _tools_ -- with sub- and argument descriptions (i.e. attached Pod values, or [declarator blocks](https://docs.raku.org/language/pod#Declarator_blocks)):

In [4]:
sink my @tool-objects =
        LLM::Tool.new(&molecular-mass),
        LLM::Tool.new(&balance-chemical-equation)
        ;

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


-----

## LLM-tool invocation example **1**

A single prompt or an array of prompts:

In [5]:
sink my $input = "What are the masses of SO2, O3, and C2H5OH? Also balance: C2H5OH + O2 = H2O + CO2";

LLM result:

In [6]:
llm-synthesize-with-tools(
        [$input, llm-prompt('NothingElse')('JSON')],
        @tool-objects,
        e => $conf41-mini, 
        form => 'JSON')

{balanced_equation => 1*C2H5OH + 3*O2 -> 2*CO2 + 3*H2O, masses => {C2H5OH => 46.069, O3 => 47.997, SO2 => 64.058}}

--- 

## "Thoughtful" response

Here is a very informative, "thoughtful" response for a quantitative Chemistry question:

In [9]:
#% markdown
my $input = "How many molecules a kilogram of water has? Use LaTeX for the formulas. (If any.)";

llm-synthesize-with-tools($input, @tool-objects)
==> { .subst(/'\[' | '\]'/, '$$', :g).subst(/'\(' | '\)'/, '$', :g) }() # Make sure LaTeX code has proper fences

To find the number of molecules in 1 kilogram of water, we use the following steps:

1. Calculate the molar mass of water (H₂O), which is approximately 18.015 g/mol.
2. Convert 1 kilogram (kg) to grams (g): $ 1 \text{ kg} = 1000 \text{ g} $.
3. Calculate the number of moles in 1000 g of water using the formula:

$$
\text{moles} = \frac{\text{mass}}{\text{molar mass}} = \frac{1000 \text{ g}}{18.015 \text{ g/mol}} \approx 55.51 \text{ mol}
$$

4. Use Avogadro's number, $ N_A = 6.022 \times 10^{23} $ molecules/mol, to find the number of molecules:

$$
\text{number of molecules} = \text{moles} \times N_A = 55.51 \times 6.022 \times 10^{23} \approx 3.34 \times 10^{25} \text{ molecules}
$$

So, 1 kilogram of water contains approximately $ 3.34 \times 10^{25} $ molecules.

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::Gemini Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).